# 1. Peridically fetch traffic trends and attack spikes from clodfare and/or IPS from abuse IPDP
# 2. Classify IPS with high DDos confidence score using machine learning
# 3, Remote IPs to coordinate and show them in fromtend globe

In [1]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')

In [2]:
import requests 

API_TOKEN = "kvVDk6bPlZ8WE305dH5EqtcMTG8tf0RkCvzJIeqL"
url="https://api.cloudflare.com/client/v4/user/tokens/verify"

headers={
    "Authorization": f"Bearer {API_TOKEN}"
}

response=requests.get(url=url,headers=headers)
# print(response.status_code)
# print(response.json())

In [3]:
API_TOKEN = "kvVDk6bPlZ8WE305dH5EqtcMTG8tf0RkCvzJIeqL"
HEADERS = {"Authorization": f"Bearer {API_TOKEN}"}

PARAMS = {
    "aggInterval": "1h",
    "dateRange": "1d",
    "format": "json"
}

def fetch_data(url):
    response = requests.get(url, headers=HEADERS, params=PARAMS)
    if response.status_code == 200:
        return response.json().get("result", {})
    else:
        print(f"Error {response.status_code}: {response.text}")
        return []

# Layer 3 URL
vector_url = "https://api.cloudflare.com/client/v4/radar/attacks/layer3/timeseries_groups/vector"
ip_url="https://api.cloudflare.com/client/v4/radar/attacks/layer3/timeseries_groups/ip_version"
protocol_url="https://api.cloudflare.com/client/v4/radar/attacks/layer3/timeseries_groups/protocol"
bitrate_url="https://api.cloudflare.com/client/v4/radar/attacks/layer3/timeseries_groups/bitrate"
duration_url="https://api.cloudflare.com/client/v4/radar/attacks/layer3/timeseries_groups/duration"

vector_data = fetch_data(vector_url)
ip_data=fetch_data(ip_url)
protocol_data=fetch_data(protocol_url)
bitrate_data=fetch_data(bitrate_url)
duration_data=fetch_data(duration_url)


df_vector = pd.DataFrame(vector_data)
df_ip = pd.DataFrame(ip_data)
df_protocol = pd.DataFrame(protocol_data)
df_bitrate = pd.DataFrame(bitrate_data)
df_duration = pd.DataFrame(duration_data)


In [20]:
to_wide=[vector_data,ip_data,protocol_data,bitrate_data,duration_data]
names=["vector_data","ip_data","protocol_data","bitrate_data","duration_data"]

In [27]:
dfs={}
for name,data in zip(names,to_wide):
    data=data["serie_0"]
    dfs[name]=pd.DataFrame(data)    

df_vector=dfs["vector_data"]
df_duration=dfs["duration_data"]
df_ip=dfs["ip_data"]
df_protocol=dfs["protocol_data"]
df_bitrate=dfs["bitrate_data"]

In [30]:
df_pr

,timestamps,IPv4,IPv6
0,2025-08-15T14:00:00Z,98.759805,1.240195
1,2025-08-15T15:00:00Z,99.417021,0.582979
2,2025-08-15T16:00:00Z,99.668995,0.331005
3,2025-08-15T17:00:00Z,99.822167,0.177833
4,2025-08-15T18:00:00Z,99.93616,0.06384
5,2025-08-15T19:00:00Z,99.919204,0.080796
6,2025-08-15T20:00:00Z,99.897969,0.102031
7,2025-08-15T21:00:00Z,99.773936,0.226064
8,2025-08-15T22:00:00Z,99.851416,0.148584
9,2025-08-15T23:00:00Z,99.750025,0.249975


In [15]:
data=vector_data["serie_0"]
df=pd.DataFrame(data)
# df_vector=df.melt(
#     id_vars=["timestamps"],
#     var_name="attack_type",
#     value_name="value"
# )

In [6]:
# data=ip_data["serie_0"]
# df=pd.DataFrame(data)
# df_ip=df.melt(
#     id_vars=["timestamps"],
#     var_name="version"
# )

In [7]:
# data=protocol_data["serie_0"]
# df=pd.DataFrame(data)
# df_protocol=df.melt(
#     id_vars=["timestamps"],
#     var_name="protocol"
# )

In [8]:
# data=bitrate_data["serie_0"]
# df=pd.DataFrame(data)
# df_bitrate=df.melt(
#     id_vars=["timestamps"],
#     var_name="bitrate"
# )


In [9]:
# data=duration_data["serie_0"]
# df=pd.DataFrame(data)
# df_duration=df.melt(
#     id_vars=["timestamps"],
#     var_name="duration"
# )


In [10]:
# df_duration
# df_bitrate
# df_protocol
# # df_ip
# # df_vector

In [11]:
# for df in [df_duration, df_bitrate, df_protocol, df_ip, df_vector]:
#     df['timestamps'] = pd.to_datetime(df['timestamps'], errors='coerce')

In [12]:
# for df in [df_vector, df_protocol, df_bitrate, df_duration, df_ip]:
#     df['timestamps'] = pd.to_datetime(df['timestamps'], utc=True)

In [13]:
# df_vector_hourly = df_vector.groupby(['attack_type', pd.Grouper(key='timestamps', freq='1H')])['value'].agg(['sum', 'mean', 'max']).reset_index()

In [16]:
df

,timestamps,Mirai (UDP) Flood,UDP Flood,TCP Flood,DNS Amplification,SYN Flood,CLDAP Flood,ACK Flood,other,DNS Flood,UDP Fragments Flood
0,2025-08-15T14:00:00Z,28.685538,15.400129,14.231487,13.940896,10.450353,8.41269,6.128595,2.378239,0.372073,0.0
1,2025-08-15T15:00:00Z,35.99098,16.441416,1.125294,19.668817,4.161313,11.82285,7.598676,2.556115,0.52558,0.108959
2,2025-08-15T16:00:00Z,39.403878,15.828123,1.216139,17.732593,1.417896,11.213858,8.034407,3.084782,0.72322,1.345103
3,2025-08-15T17:00:00Z,24.2208,7.531281,0.536785,36.820215,0.492032,8.412436,5.068454,1.12956,0.72299,15.065446
4,2025-08-15T18:00:00Z,11.933159,5.016417,0.245627,44.406814,0.295244,4.247847,2.520854,0.763497,0.377305,30.193236
5,2025-08-15T19:00:00Z,37.051305,8.02472,9.501916,19.524955,4.483426,6.415199,4.23901,2.464873,2.219082,6.075514
6,2025-08-15T20:00:00Z,26.777622,5.462247,0.255107,24.971758,0.188849,15.494455,2.457407,0.789503,1.467557,22.135495
7,2025-08-15T21:00:00Z,54.789033,18.503945,0.590702,10.115247,0.339187,8.051306,4.417579,2.456289,0.291453,0.445258
8,2025-08-15T22:00:00Z,55.548273,8.947946,1.165113,10.087433,2.245069,6.21937,4.896037,9.886993,0.737443,0.266322
9,2025-08-15T23:00:00Z,43.160641,15.329229,0.9477,14.898446,1.224446,11.196267,10.168078,2.4793,0.363317,0.232576
